<a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso-data-management/blob/main/tutorials/04_Upload_frames_to_Zooniverse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">Colab KSO Tutorials #4: Upload frames to Zooniverse</h1>
<h3 align="right">Written by KSO Team</h3>

This notebook takes you through the process of: 
* Connecting to a Zooniverse project
* Retrieving the classifications/annotations that citizens have provided on the videos, using aggregation settings. To experiment with these aggregation settings, see Notebook 8. 
* Cutting the videos into images from the moment the species of interest was seen for the first time
* Modify these images with for example a color correction
* Upload the images to Zooniverse for the 2nd part of the workflow.

If you do not have a project with us yet, you can run the template project to get a taste of how it all works. Only the uploading of the images to Zooniverse will not be possible if you are not a member of our template project on Zooniverse.

🔴 <span style="color:red">&nbsp;NOTE: If you want to run another project than the template project, you need to have a Zooniverse account and be a member of the corresponding project.  </span>

# Set up and requirements

## Install kso_data_management and its requirements

In [ ]:
# @title <font size="5">↓ ឵឵<i>Install kso_data_management and its requirements</font> { vertical-output: true }

from IPython.display import clear_output

try:
    import google.colab
    import os

    IN_COLAB = True
    print("Running in Colab...")

    # Clone kso-data-management repo
    !git clone --quiet --recurse-submodules -b main https://github.com/ocean-data-factory-sweden/kso-data-management.git
    %pip install -q --upgrade pip
    %pip install -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/requirements.txt)
    %pip install -qr <(sed '/boto3/d;/ipywidgets/d' kso-data-management/kso_utils/requirements.txt)

    # Fix libmagic issue
    !apt-get -qq update && apt-get -qq install -y libmagic-dev > /dev/null

    # Enable external widgets
    from google.colab import output

    output.enable_custom_widget_manager()

    os.chdir("kso-data-management/tutorials")
    try:
        clear_output()
        print("All packages are installed and ready to go!")
    except:
        clear_output()
        print("There have been some issues installing the packages!")
except:
    IN_COLAB = False

    # Install requirements
    #%pip install -q --no-warn-script-location --upgrade pip --user
    #%pip install -qr ../requirements.txt --user
    #%pip install -qr ../kso_utils/requirements.txt --user

    !jupyter nbextension install --user --py widgetsnbextension
    !jupyter nbextension enable --user --py widgetsnbextension
    !jupyter nbextension install --user --py jupyter_bbox_widget
    !jupyter nbextension enable --user --py jupyter_bbox_widget
    !jupyter nbextension install --user --py ipysheet
    !jupyter nbextension enable --user --py ipysheet

    clear_output()
    print("Running locally... you're good to go!")

### Import Python packages

In [ ]:
# @title <font size="5">↓ ឵឵<i>Import Python packages</font> { vertical-output: true }

# Set the directory of the libraries
import sys

try:
    if "kso_utils" not in sys.modules:
        sys.path.append("..")
        import kso_utils.kso_utils

        sys.modules["kso_utils"] = kso_utils.kso_utils
        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Import required modules
import kso_utils.tutorials_utils as t_utils
import kso_utils.t4_utils as t4
import kso_utils.project_utils as p_utils
from kso_utils.project import ProjectProcessor


print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = t_utils.choose_project()

In [ ]:
# Find project
project = p_utils.find_project(project_name=project_name.value)
# Initialise pp
pp = ProjectProcessor(project)

## Retrieve info about zooniverse clips

In [ ]:
pp.get_zoo_info()

## Get frames from ZU

In [ ]:
# Get all available frames for the selected species from clips
pp.get_frames()

In [ ]:
# Generate suitable frames for upload by modifying initial frames
pp.generate_zu_frames()

## Get frames from movies (Manual)

In [ ]:
# Choose folder containing movies
input_folder = t4.choose_folder()

In [ ]:
# Choose output folder for frames
output_folder = t4.choose_folder()

In [ ]:
# Generate suitable frames for upload by modifying initial frames
pp.generate_custom_frames(
    input_folder.selected, output_folder.selected, num_frames=10, frames_skip=None
)

In [ ]:
# Review the size of the modified clips
t4.check_frame_size(frame_paths=pp.generated_frames["modif_frame_path"].unique())

### Preview the frames

In [ ]:
# Compare the original and modified clips
t4.compare_frames(df=pp.generated_frames)

### Upload frames to Zooniverse

You may receive an error message related to file size if clips exceed the recommended limit for Zooniverse uploads. In this case, we recommend shortening the clip length to achieve a suitable filesize.

🔴 <span style="color:red">&nbsp;NOTE: If you run the template project without being a member of our template project, it is not possible to run this last cell.  </span>

In [ ]:
pp.upload_zu_subjects(pp.generated_frames, "frame")

In [ ]:
# END